In [1]:
!pip install datasets
!pip install wandb
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
import evaluate
import wandb
import torchaudio
from datasets import load_dataset, DatasetDict, Audio
from sklearn.model_selection import train_test_split
import random
from IPython.display import Audio, display
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer

In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hildurdav (hildurdav-kth-royal-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import os
import pandas as pd
from datasets import load_dataset, Audio
from sklearn.model_selection import train_test_split

# Step 1: Get File Paths and Labels
directory = "/content/"  # Ensure this is correct
files = [f for f in os.listdir(directory) if f.endswith(".wav")]
print("Found files:", files)

df = pd.DataFrame({
    "file": files,  # Use only filenames, not full paths
    "label": [f.split("_")[0] for f in files]
})

# Step 2: Split into Train and Validation Sets
train_df, eval_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# Step 3: Save CSV Files
train_csv = "train.csv"
eval_csv = "eval.csv"
train_df.to_csv(train_csv, index=False)
eval_df.to_csv(eval_csv, index=False)

# Step 4: Load Dataset from CSV
dataset = load_dataset("csv", data_files={"train": train_csv, "validation": eval_csv})

# Step 5: Set `data_dir` and Cast Audio
dataset = dataset.cast_column("file", Audio(sampling_rate=16000))

# Verify Dataset
print(dataset)


Found files: ['Norrland_72.wav', 'Norrland_53.wav', 'Gotaland_139.wav', 'Norrland_79.wav', 'Gotaland_117.wav', 'Gotaland_143.wav', 'Norrland_83.wav', 'Gotaland_125.wav', 'Norrland_73.wav', 'Gotaland_135.wav', 'Norrland_82.wav', 'Gotaland_136.wav', 'Norrland_84.wav', 'Gotaland_142.wav', 'Gotaland_121.wav', 'Norrland_61.wav', 'Norrland_78.wav', 'Gotaland_127.wav', 'Norrland_68.wav', 'Norrland_70.wav', 'Gotaland_134.wav', 'Gotaland_124.wav', 'Norrland_67.wav', 'Norrland_59.wav', 'Norrland_80.wav', 'Gotaland_130.wav', 'Gotaland_131.wav', 'Gotaland_122.wav', 'Norrland_56.wav', 'Norrland_81.wav', 'Norrland_69.wav', 'Gotaland_138.wav', 'Gotaland_144.wav', 'Gotaland_120.wav', 'Gotaland_141.wav', 'Norrland_63.wav', 'Gotaland_137.wav', 'Norrland_74.wav', 'Gotaland_140.wav', 'Norrland_75.wav', 'Norrland_60.wav', 'Gotaland_132.wav', 'Norrland_57.wav', 'Norrland_76.wav', 'Norrland_62.wav', 'Gotaland_133.wav', 'Gotaland_118.wav', 'Gotaland_123.wav', 'Norrland_77.wav', 'Gotaland_128.wav', 'Norrland_6

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'label'],
        num_rows: 48
    })
    validation: Dataset({
        features: ['file', 'label'],
        num_rows: 12
    })
})


In [5]:
def get_labels(df):
    labels = list(set(df['label']))
    label2id = {label: i for i, label in enumerate(labels)}
    id2label = {i: label for label, i in label2id.items()}
    return labels, label2id, id2label

labels, label2id, id2label = get_labels(df)

In [6]:
model_checkpoint = "facebook/wav2vec2-large-xlsr-53"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [7]:
max_duration = 10.0  # seconds
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["file"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    # Add label conversion to numerical IDs using label2id
    inputs["labels"] = [label2id[x] for x in examples["label"]]
    return inputs

In [8]:
# Update the dataset mapping to remove 'file' and 'label'
encoded_dataset = dataset.map(preprocess_function, remove_columns=["file", "label"], batched=True)
encoded_dataset

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_values', 'attention_mask', 'labels'],
        num_rows: 48
    })
    validation: Dataset({
        features: ['input_values', 'attention_mask', 'labels'],
        num_rows: 12
    })
})

In [9]:
print(len(encoded_dataset["train"][0]['input_values']))

160000


In [10]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-dialect-classifier",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to="wandb"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}

In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

<ipython-input-15-c12e664a5c74>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693188,0.500000


OutOfMemoryError: CUDA out of memory. Tried to allocate 78.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 62.12 MiB is free. Process 8800 has 14.68 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 107.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)